### Loading the libraries

In [1]:
%classpath add mvn com.salesforce.transmogrifai transmogrifai-core_2.11 0.5.1

In [2]:
%classpath add mvn org.apache.spark spark-mllib_2.11 2.3.0

### Import the classes

In [3]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf

import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.stages.impl.classification._
import com.salesforce.op.evaluators.Evaluators

import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.sql.functions.udf
import com.salesforce.op._
import com.salesforce.op.features._
import com.salesforce.op.features.types._
import com.salesforce.op.stages.impl.classification._
import com.salesforce.op.evaluators.Evaluators


In [4]:
import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders

import com.salesforce.op.OpWorkflow
import com.salesforce.op.evaluators.Evaluators
import com.salesforce.op.readers.DataReaders


### Instantiate Spark Instance

In [5]:
val conf = new SparkConf().setMaster("local[*]").setAppName("TitanicPrediction")
implicit val spark = SparkSession.builder.config(conf).getOrCreate()

org.apache.spark.sql.SparkSession@149f9714

### Schema and Feature Creation

In [6]:
case class SimpleRegression (
  population: Double,
  profit: Double)

defined class SimpleRegression


In [7]:
val population = FeatureBuilder.RealNN[SimpleRegression].extract(_.population.toRealNN).asPredictor
val profit = FeatureBuilder.RealNN[SimpleRegression].extract(_.profit.toRealNN).asResponse

Feature(name = profit, uid = RealNN_000000000002, isResponse = true, originStage = FeatureGeneratorStage_000000000002, parents = [], distributions = [])

#### Load the data

In [8]:
import spark.implicits._

val trainFilePath = "../src/main/resources/SimpleRegressionDataset/simple_regression.csv"
val trainDataReader = DataReaders.Simple.csvCase[SimpleRegression](
    path = Option(trainFilePath)
)

org.apache.spark.sql.SparkSession$implicits$@31be64a3

In [9]:
// check that path exists
scala.reflect.io.File(trainFilePath).exists

true

In [10]:
import com.salesforce.op.stages.impl.tuning.{DataCutter, DataSplitter}
val features = Seq(population).transmogrify()
val randomSeed = 42L
val splitter = DataSplitter(seed = randomSeed)

DataSplitter_000000000005

#### Model Selector
The ModelSelector is an Estimator that uses data to find the best model. 

In [11]:
import com.salesforce.op.stages.impl.regression.RegressionModelsToTry.{OpGBTRegressor, OpRandomForestRegressor,OpLinearRegression}
import com.salesforce.op.stages.impl.regression.RegressionModelSelector

val cutter = DataCutter(reserveTestFraction = 0.2, seed = randomSeed)

val prediction = RegressionModelSelector
      .withCrossValidation(
        dataSplitter = Some(splitter), seed = randomSeed,
        //modelTypesToUse = Seq(OpGBTRegressor, OpRandomForestRegressor)
          modelTypesToUse = Seq(OpLinearRegression)
        
 
).setInput(profit, features).getOutput()

Feature(name = population-profit_3-stagesApplied_Prediction_00000000000f, uid = Prediction_00000000000f, isResponse = true, originStage = ModelSelector_00000000000f, parents = [RealNN_000000000002,OPVector_000000000004], distributions = [])

#### Evaluators and Workflow
Factory that performs the evaluation of metrics for regression. The metrics returned are rmse, mse, r2 and mae.
* Mean Squared Error (MSE)	
* Root Mean Squared Error (RMSE)	
* Mean Absolute Error (MAE)	
* Coefficient of Determination 

OpWorkflows create and transform the raw data needed to compute Features fed into them. In addition they optimize the application of Stages needed to create the final Features ensuring optimal computations within the full pipeline DAG. OpWorkflows can be fit to a given dataset using the .train() method. This produces an OpWorkflowModel which can then be saved to disk and applied to another dataset.

In [12]:
val evaluator = Evaluators.Regression().setLabelCol(profit).
      setPredictionCol(prediction)

OpRegressionEvaluator_000000000010

In [13]:
val workflow = new OpWorkflow().setResultFeatures(prediction, profit).setReader(trainDataReader)
val workflowModel = workflow.train()

com.salesforce.op.OpWorkflowModel@1075401c

#### Score and evaluate

In [14]:
val dfScoreAndEvaluate = workflowModel.scoreAndEvaluate(evaluator)
val dfScore = dfScoreAndEvaluate._1//.withColumnRenamed("population-profit_3-stagesApplied_Prediction_000000000011","predicted_profit")
val dfEvaluate = dfScoreAndEvaluate._2
println("Evaluate:\n" + dfEvaluate.toString())

dfScore.show(false)

Evaluate:
{
  "RootMeanSquaredError" : 2.9932433355081214,
  "MeanSquaredError" : 8.959505665563784,
  "R2" : 0.7018464316796386,
  "MeanAbsoluteError" : 2.2003839379298773
}
+--------------------+-------+---------------------------------------------------------+
|key                 |profit |population-profit_3-stagesApplied_Prediction_00000000000f|
+--------------------+-------+---------------------------------------------------------+
|-6995599919488981260|17.592 |[prediction -> 3.4381831736669186]                       |
|-7944605230946576675|9.1302 |[prediction -> 2.7546201780689543]                       |
|-5201368887788792308|13.662 |[prediction -> 6.265040101719282]                        |
|3331366171450530671 |11.854 |[prediction -> 4.486414820934333]                        |
|-1241173145884514893|6.8233 |[prediction -> 3.1444060139859937]                       |
|-7995239958613860985|11.886 |[prediction -> 6.105768984784798]                        |
|2106318929888492848 |4.

null